In [1]:
import numpy as np
import scipy.misc
from scipy.fftpack import dct, idct
import sys
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import random
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.fftpack import dct, idct
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import sympy

%matplotlib inline


class ImageLoader:
    def __init__(self, FILE_PATH):
        self.img = np.array(Image.open(FILE_PATH))
        # 行数
        self.row_blocks_count = self.img.shape[0] // 8
        # 列数
        self.col_blocks_count = self.img.shape[1] // 8

    def get_points(self, POINT):
        Row = random.randint(0, len(self.img) - POINT - 1)
        Col = random.randint(0, len(self.img) - 1)
        return self.img[Row : Row + POINT, Col]

    def get_block(self, col, row):
        return self.img[col * 8 : (col + 1) * 8, row * 8 : (row + 1) * 8]


# plt.rcParams['font.family'] ='sans-serif'#使用するフォント
# plt.rcParams["font.sans-serif"] = "Source Han Sans"
plt.rcParams["font.family"] = "Source Han Sans JP"  # 使用するフォント
plt.rcParams["xtick.direction"] = "in"  # x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["ytick.direction"] = "in"  # y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["xtick.major.width"] = 1.0  # x軸主目盛り線の線幅
plt.rcParams["ytick.major.width"] = 1.0  # y軸主目盛り線の線幅
plt.rcParams["font.size"] = 12  # フォントの大きさ
plt.rcParams["axes.linewidth"] = 1.0  # 軸の線幅edge linewidth。囲みの太さ
matplotlib.font_manager._rebuild()

MONO_DIR_PATH = "../../Mono/"
AIRPLANE = ImageLoader(MONO_DIR_PATH + "airplane512.bmp")
BARBARA = ImageLoader(MONO_DIR_PATH + "barbara512.bmp")
BOAT = ImageLoader(MONO_DIR_PATH + "boat512.bmp")
GOLDHILL = ImageLoader(MONO_DIR_PATH + "goldhill512.bmp")
LENNA = ImageLoader(MONO_DIR_PATH + "lenna512.bmp")
MANDRILL = ImageLoader(MONO_DIR_PATH + "mandrill512.bmp")
MILKDROP = ImageLoader(MONO_DIR_PATH + "milkdrop512.bmp")
SAILBOAT = ImageLoader(MONO_DIR_PATH + "sailboat512.bmp")

IMAGES = [
    AIRPLANE,
    BARBARA,
    BOAT,
    GOLDHILL,
    LENNA,
    MANDRILL,
    MILKDROP,
    SAILBOAT
]

In [2]:
class DMLCT:
    def __init__(self, n_bar, N):
        self.n_bar = n_bar
        self.N = N

        self.x_l = (2 * np.arange(N) + 1) / (2 * N)
        self.s_l = np.arange(n_bar) / (n_bar - 1)
        self.xi = (np.arange(n_bar + 1) - 0.5) / (n_bar - 1)

        self.lambda_kh = self.get_lambda_kh(self.n_bar)

        self.w_k_j = self.get_w_k_j(self.n_bar, self.N)
        self.W_L_k_kh = self.get_W_L_k_kh(self.n_bar, self.N)
        self.W_k_kh = self.get_W_k_kh(self.n_bar, self.N)
        self.W_R_k_kh = self.get_W_R_k_kh(self.n_bar, self.N)

    def Lagrange_j(self, j):
        x = sympy.Symbol("x")
        L_x = 1.0
        for l in range(self.n_bar):
            if l != j:
                L_x *= (x - self.s_l[l]) / (self.s_l[j] - self.s_l[l])
        return sympy.integrate(L_x)

    def get_lambda_kh(self, n_bar):
        lambda_kh = np.ones(n_bar)
        lambda_kh[0] = np.sqrt(1 / 2)
        return lambda_kh

    def get_w_k_j(self, n_bar, N):
        L_j = np.zeros((n_bar, N))
        x = sympy.Symbol("x")
        for j in range(n_bar):
            temp = []
            Lj = self.Lagrange_j(j)
            for k in range(N):
                temp.append(Lj.subs(x, self.x_l[k]))
            L_j[j] = np.array(temp)

        w_k_j = np.zeros((n_bar, N))
        for j in range(n_bar):
            w_k_j[j] = scipy.fftpack.dct(L_j[j], norm="ortho")
        return w_k_j

    def get_W_L_k_kh(self, n_bar, N):
        W_L_k_kh = np.zeros((n_bar - 1, N))
        lambda_kh = self.get_lambda_kh(n_bar)

        for kh in range(n_bar - 1):
            W_L_k_kh[kh] = (
                (1 - n_bar)
                * np.sqrt(2 / N)
                * lambda_kh[kh]
                * np.cos(np.pi * kh * (self.xi[0] + 1))
                * self.w_k_j[0]
            )
        return W_L_k_kh

    def get_W_k_kh(self, n_bar, N):
        W_k_kh = np.zeros((n_bar - 1, N))
        for kh in range(n_bar - 1):
            sum_sin = np.zeros(N)
            for j in range(1, n_bar - 2 + 1):
                sum_sin += np.sin(np.pi * kh * self.s_l[j]) * self.w_k_j[j]

            W_k_kh[kh] = (
                (n_bar - 1)
                * np.sqrt(2 / N)
                * self.lambda_kh[kh]
                * (
                    np.cos(np.pi * kh * self.xi[1])
                    * (self.w_k_j[0] - (-1) ** (kh) * self.w_k_j[n_bar - 1])
                    - 2 * np.sin((np.pi * kh) / (2 * (n_bar - 1))) * sum_sin
                )
            )
        return W_k_kh

    def get_W_R_k_kh(self, n_bar, N):
        W_R_k_kh = np.zeros((n_bar - 1, N))
        for kh in range(n_bar - 1):
            W_R_k_kh[kh] = (
                (n_bar - 1)
                * np.sqrt(2 / N)
                * self.lambda_kh[kh]
                * np.cos(np.pi * kh * (self.xi[n_bar] - 1))
                * self.w_k_j[n_bar - 1]
            )
        return W_R_k_kh

In [3]:
def get_F_L_k_horizontal(arr, N, row, col):
    # w
    if col == 0:
        w_block = np.zeros(N)
    else:
        w_block = arr[row, (col - 1) * N : col * N]

    return w_block

In [4]:
def get_F_R_k_horizontal(arr, N, row, col):
    # e
    if col == arr.shape[1] // N - 1:
        e_block = np.zeros(N)
    else:
        e_block = arr[row, (col + 1) * N : (col + 2) * N]

    return e_block

In [5]:
def get_F_L_k_vertical(arr, N, row, col):
    # n
    if row == 0:
        n_block = np.zeros(N)
    else:
        n_block = arr[(row - 1) * N : row * N, col]

    return n_block

In [6]:
def get_F_R_k_vertical(arr, N, row, col):
    # s
    if row == arr.shape[0] // N - 1:
        s_block = np.zeros(N)
    else:
        s_block = arr[(row + 1) * N : (row + 2) * N, col]

    return s_block

In [9]:
n_bar = 4
N = 8

In [10]:
dmlct = DMLCT(n_bar, N)

In [11]:
IMG = LENNA

In [12]:
Fk = np.zeros(IMG.img.shape)

# 順変換

In [13]:
def DMLCT_forward(IMG,n_bar,N):
    Fk = np.zeros(IMG.img.shape)
    ## 縦方向
    ### DCT
    for row in range(IMG.img.shape[0] // N):
        for col in range(IMG.img.shape[1]):
            eight_points = IMG.img[N * row : N * (row + 1), col]
            c = scipy.fftpack.dct(eight_points, norm="ortho")
            Fk[N * row : N * (row + 1), col] = c
    ### 残差
    dmlct = DMLCT(n_bar,N)
    for row in range(IMG.img.shape[0] // N):
        for col in range(IMG.img.shape[1]):
            # ビューなら直接いじっちゃう
            F = Fk[N * row : N * (row + 1), col]
            F_L = get_F_L_k_vertical(Fk, N, row, col)
            F_R = get_F_R_k_vertical(Fk, N, row, col)

            U_k_n_bar = np.zeros(N)
            for kh in range(n_bar - 2 + 1):
                U_k_n_bar += (
                    F_L[kh] * dmlct.W_L_k_kh[kh]
                    + F[kh] * dmlct.W_k_kh[kh]
                    + F_R[kh] * dmlct.W_R_k_kh[kh]
                )

            # n_bar = 4 なら 0,1,2は残す 3,4,5,6,7を書き換える
            F[n_bar - 2 + 1 :] -= U_k_n_bar[n_bar - 2 + 1 :]
    # 0を残す
    for k in reversed(range(1, n_bar - 2 + 1)):
        dmlct = DMLCT(k+1, N)
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1]):
                # ビューなら直接いじっちゃう
                F = Fk[N * row : N * (row + 1), col]
                F_L = get_F_L_k_vertical(Fk, N, row, col)
                F_R = get_F_R_k_vertical(Fk, N, row, col)

                U_k_n_bar = np.zeros(N)

                for kh in range((k + 1) - 2 + 1):
                    U_k_n_bar += (
                        F_L[kh] * dmlct.W_L_k_kh[kh]
                        + F[kh] * dmlct.W_k_kh[kh]
                        + F_R[kh] * dmlct.W_R_k_kh[kh]
                    )

                F[k] -= U_k_n_bar[k]
    ## 横方向
    ### DCT
    for row in range(Fk.shape[0]):
        for col in range(Fk.shape[1] // N):
            eight_points = Fk[row, N * col : N * (col + 1)]
            c = scipy.fftpack.dct(eight_points, norm="ortho")
            Fk[row, N * col : N * (col + 1)] = c
            
    ### 残差
    dmlct = DMLCT(n_bar,N)
    for row in range(IMG.img.shape[0]):
        for col in range(IMG.img.shape[1] // N):
            F = Fk[row, N * col : N * (col + 1)]
            F_L = get_F_L_k_horizontal(Fk, N, row, col)
            F_R = get_F_R_k_horizontal(Fk, N, row, col)

            U_k_n_bar = np.zeros(N)
            for kh in range(n_bar - 2 + 1):
                U_k_n_bar += (
                    F_L[kh] * dmlct.W_L_k_kh[kh]
                    + F[kh] * dmlct.W_k_kh[kh]
                    + F_R[kh] * dmlct.W_R_k_kh[kh]
                )

            # n_bar = 4 なら 0,1,2は残す 3,4,5,6,7を書き換える
            F[n_bar - 2 + 1 :] -= U_k_n_bar[n_bar - 2 + 1 :]
    # 0を残す
    for k in reversed(range(1, n_bar - 2 + 1)):
        dmlct = DMLCT(k+1, N)
        for row in range(IMG.img.shape[0]):
            for col in range(IMG.img.shape[1] // N):
                F = Fk[row, N * col : N * (col + 1)]
                F_L = get_F_L_k_horizontal(Fk, N, row, col)
                F_R = get_F_R_k_horizontal(Fk, N, row, col)

                U_k_n_bar = np.zeros(N)

                for kh in range((k + 1) - 2 + 1):
                    U_k_n_bar += (
                        F_L[kh] * dmlct.W_L_k_kh[kh]
                        + F[kh] * dmlct.W_k_kh[kh]
                        + F_R[kh] * dmlct.W_R_k_kh[kh]
                    )

                F[k] -= U_k_n_bar[k]
    
    return Fk

In [37]:
Fk_values = np.zeros((512,512))

In [38]:
for IMG in tqdm_notebook(IMAGES):
    values = np.zeros((25,4))
    Fk = np.zeros(IMG.img.shape)
    for row in range(IMG.img.shape[0] // N):
        for col in range(IMG.img.shape[1] // N):
            block = IMG.img[row * N : (row + 1) * N, col * N : (col + 1) * N]
            c = scipy.fftpack.dct(
                scipy.fftpack.dct(block, axis=0, norm="ortho"), axis=1, norm="ortho"
            )
            Fk[row * N : (row + 1) * N, col * N : (col + 1) * N] = c

    Fk_values += np.abs(Fk)

In [39]:
Fk_values /= len(IMAGES)
pd.DataFrame(Fk_values).to_csv("DCT_coef_ave.csv",header=False,index=False)

In [31]:
Vk_values = np.zeros((512,512))

In [32]:
for n_bar in tqdm_notebook(range(2,6)):
    Vk_values = np.zeros((512,512))
    for IMG in IMAGES:
        Fk = DMLCT_forward(IMG,n_bar,N)
        Vk_values += np.abs(Fk)
    
    pd.DataFrame(Vk_values / len(IMAGES)).to_csv("DMLCT_" + str(n_bar) + "_coef_ave.csv",header=False,index=False)

In [12]:
Fk_values = pd.read_csv("lena_coef.csv",header=None)
Fk_values.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,1283.500000,4.768282,3.240387,-0.199192,0.250000,-0.553887,-4.589386,5.638452,1270.375000,17.630173,...,-11.455640,-4.018132,1087.625000,80.231033,-79.736328,11.407176,-32.375000,-1.795734,-11.214910,-1.037048
1,7.947236,-0.787859,0.554729,-4.932320,1.960236,2.978430,-3.797149,3.322186,11.923947,-2.692679,...,-0.979841,0.288014,213.425125,-65.261223,-37.442665,1.817699,-20.977341,-5.145244,-4.907657,-5.205546
2,-5.034956,-0.297405,-1.551775,1.724977,-0.676499,-0.452498,1.849877,-2.200248,-4.887996,2.336387,...,-0.176776,-0.081572,-127.513046,27.490141,29.562634,-1.004208,11.698216,2.233491,1.381281,3.510292
3,2.261879,1.139027,1.703857,0.924223,-0.760606,-1.368626,0.214283,1.142231,-1.152210,-1.786861,...,1.021881,-0.148397,41.115940,7.694039,-19.010632,-0.721418,-2.766168,0.152978,1.820599,-2.324284
4,-1.000000,-1.149702,-0.343048,-1.359623,1.750000,1.118855,-1.481490,-0.672886,2.625000,1.142018,...,-0.923880,0.304489,5.625000,-21.595932,8.669580,3.123973,-1.375000,-0.521765,-2.914560,2.504384


In [14]:
Vk_values = pd.read_csv("DMLCT_lenna_ashi_coef.csv",header=None)
Vk_values.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,NaN,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,...,502.000000,503.000000,504.000000,505.000000,506.000000,507.000000,508.000000,509.000000,510.000000,511.000000
1,0.0,1283.500000,5.225678,3.435357,-0.649333,0.420090,-0.678857,-4.545228,5.638457,1270.375000,...,-11.106677,-4.018135,1087.625000,84.600404,-75.553595,18.759157,-30.640868,-0.029306,-10.760816,-1.037052
2,1.0,9.053840,-0.912347,0.863148,-5.012748,1.996344,2.892838,-3.942203,3.496862,12.542530,...,-0.867339,0.164506,237.323995,-54.479366,-35.253852,6.692903,-20.595459,-4.030427,-4.980824,-5.506639
3,2.0,-5.838662,-0.384504,-1.798490,1.674371,-0.540735,-0.346868,1.913992,-2.317477,-5.087883,...,-0.304638,-0.070194,-142.079673,20.520344,28.568535,-4.328481,11.438158,1.499079,1.567293,3.712079
4,3.0,2.915586,1.402300,1.846421,0.982653,-0.865084,-1.454022,0.077189,1.301766,-0.959492,...,1.055040,-0.143198,53.995480,10.002001,-20.368213,1.127818,-3.143366,0.494088,1.636535,-2.611615


In [11]:
np.abs(Vk_values) / np.abs(Fk_values)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,NaN,0.000000,0.308605,10.040580,12.000000,7.221689,1.089470,1.064122,0.005510,0.453768,...,124.933684,0.462476,6.281859,6.333374,44.358043,15.660232,282.892708,45.386010,491.780750,NaN
1,0.000000,1629.098438,9.420230,0.696499,0.331253,0.141044,0.178781,1.368144,0.472868,471.788518,...,38.563014,0.018827,16.665716,2.259465,41.565516,0.894258,5.955183,0.005971,2.067183,NaN
2,0.198611,30.442763,0.587938,0.500382,7.409844,4.411831,1.563801,1.791708,0.715398,5.368344,...,10.632786,0.001290,8.633058,1.842845,35.106110,0.572130,9.221194,2.917891,1.418920,NaN
3,0.884220,5.126009,0.225667,1.945948,2.201363,0.395094,1.618742,1.675661,2.011333,2.847386,...,2.052861,0.001707,18.466202,1.079414,39.600503,1.564793,74.770130,0.823399,0.674312,NaN
4,3.000000,2.535950,4.087764,1.358039,0.561516,0.773187,0.981459,0.114713,0.495911,0.840173,...,3.464953,0.025457,2.500262,1.153689,6.519972,0.820232,6.024488,0.169524,0.653468,NaN
5,3.303954,2.753886,0.754678,3.336049,0.755584,2.609613,0.697712,1.924640,38.427481,4.859710,...,2.554345,0.037936,0.114548,22.127221,1.818182,4.629893,1.604744,0.340433,0.781817,NaN
6,2.936270,5.241604,0.176311,1.079224,0.031780,0.905752,0.577812,1.843059,0.264531,0.003816,...,0.186046,0.044474,23.239121,5.373129,0.323314,2.847578,0.189937,2.501528,0.457368,NaN
7,4.525188,4.256403,0.120832,1.908101,1.662641,0.821519,4.067259,1.838123,0.301612,265.035878,...,4.653856,0.022203,1.566137,0.152374,0.716989,2.619839,0.967526,5.281742,0.108860,NaN
8,0.005592,0.273318,0.094432,6.922049,3.003131,0.559069,5.544934,0.895592,0.000522,0.550908,...,1.438775,0.000369,0.612386,0.244848,0.193214,0.375076,0.475225,0.415505,0.047332,NaN
9,2.969241,660.569098,1.259261,3.333186,0.088730,1.424935,35.583374,0.237798,0.180814,391.647466,...,0.815419,0.021952,11.349434,0.400103,2.983437,2.247921,1.983567,0.552626,2.000976,NaN
